In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
!mkdir /content/data
%cd /content/data
#load dataset
gdd.download_file_from_google_drive(file_id='1pYDib6RtXkddBPmDrhGQtUdoH-9QXdfB',
                                    dest_path='./fish.zip',
                                    unzip=True)

/content/data
Unzipping...Done.


In [ ]:
%cd /content
# mount at google drive
# you are not in colab enviro please ignore this part
from google.colab import drive
from google.colab import files
import os
drive.mount('/content/drive')


/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/VRDL/final_project3
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from utils import dataset
from utils import function as ft
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.transforms import InterpolationMode
torch.manual_seed(0)

/content/drive/MyDrive/VRDL/final_project3


# split dataset

In [ ]:
!python /content/drive/MyDrive/VRDL/final_project3/utils/split_train_val.py

# data augmentation

In [ ]:
train_trans = transforms.Compose([
            transforms.Resize((400, 400), InterpolationMode.BILINEAR),
            transforms.RandomCrop((299, 299)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0, hue=0),
            
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0), 
])
val_trans = transforms.Compose([
            transforms.Resize((400, 400), InterpolationMode.BILINEAR),
            transforms.CenterCrop((299, 299)),

            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)


In [ ]:
BATCHSIZE = 8
TRAIN_PATH = '/content/data/fish/train_split'
VAL_PATH = '/content/data/fish/val_split'
TEST_PATH = '/content/data/fish/test'

trainset = dataset.fish(TRAIN_PATH, is_train=True, transform=train_trans)
valset = dataset.fish(VAL_PATH, is_train=False, transform=val_trans)

train_loader = DataLoader(dataset = trainset,
                          batch_size = BATCHSIZE,
                          shuffle = True,
                          num_workers = 4,
                          pin_memory = True)
val_loader = DataLoader(dataset = valset,
                        batch_size = BATCHSIZE,
                        shuffle = False,
                        num_workers = 4,
                        pin_memory = True)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# resnet152

In [ ]:
r152 = models.resnet152(pretrained=True)
for param in r152.parameters():
    param.requires_grad = True
r152.fc = nn.Linear(2048, 8)
r152.load_state_dict(torch.load('/content/drive/MyDrive/VRDL/final_project3/resnet152/sgd0.0001_cos_loss_229.pth'))

opt = optim.SGD(filter(lambda p: p.requires_grad, r152.parameters()), lr=0.0001)
sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=6, verbose=True)

CONFIG = {
    'model':r152,
    'optimizer':opt,
    'scheduler':sch,
    'train_loader':train_loader,
    'val_loader':val_loader,
    'EPOCHS':40,
    'MODELPATH':'/content/drive/MyDrive/VRDL/final_project3/resnet152/sgd0.0001_cos_loss_229.pth'
}

ft.train(**CONFIG)

Adjusting learning rate of group 0 to 1.0000e-04.


100%|██████████| 378/378 [00:55<00:00,  6.84it/s]

Adjusting learning rate of group 0 to 9.3301e-05.


epoch 1%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0792   acc: 77.74%
val set=====>loss: 0.0328   acc: 92.74%



100%|██████████| 378/378 [00:54<00:00,  6.95it/s]

...

Adjusting learning rate of group 0 to 5.0000e-05.


epoch 39%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0198   acc: 94.27%
val set=====>loss: 0.0354   acc: 95.12%



100%|██████████| 378/378 [00:54<00:00,  6.95it/s]

Adjusting learning rate of group 0 to 2.5000e-05.


epoch 40%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.0179   acc: 94.87%
val set=====>loss: 0.0347   acc: 95.25%



# regnet_y_32gf

In [ ]:
reg = models.regnet_y_32gf(pretrained=True)
for param in reg.parameters():
    param.requires_grad = True
reg.fc = nn.Linear(3712, 8)

opt = optim.SGD(filter(lambda p: p.requires_grad, reg.parameters()), lr=0.0001)
sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=6, verbose=True)

CONFIG = {
    'model':reg,
    'optimizer':opt,
    'scheduler':sch,
    'train_loader':train_loader,
    'val_loader':val_loader,
    'EPOCHS':40,
    'MODELPATH':'/content/drive/MyDrive/VRDL/final_project3/regnet_y_32gf/sgd0.001_cos_loss265.pth'
}

ft.train(**CONFIG)

# efficientnet_b7

In [ ]:
eff = models.efficientnet_b7(pretrained=True)
for param in eff.parameters():
    param.requires_grad = True
eff.classifier = nn.Linear(2560, 8)

opt = optim.SGD(filter(lambda p: p.requires_grad, eff.parameters()), lr=0.0001)
sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=6, verbose=True)

CONFIG = {
    'model':eff,
    'optimizer':opt,
    'scheduler':sch,
    'train_loader':train_loader,
    'val_loader':val_loader,
    'EPOCHS':40,
    'MODELPATH':'/content/drive/MyDrive/VRDL/final_project3/efficientnet_b7/sgd0.0001_cos_loss265.pth'
}

ft.train(**CONFIG)

Adjusting learning rate of group 0 to 1.0000e-04.


100%|██████████| 755/755 [02:25<00:00,  5.20it/s]

Adjusting learning rate of group 0 to 9.3301e-05.


epoch 1%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.5116   acc: 22.26%
val set=====>loss: 0.5219   acc: 13.19%



100%|██████████| 755/755 [02:12<00:00,  5.69it/s]

...

Adjusting learning rate of group 0 to 5.0000e-05.


epoch 39%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.3439   acc: 52.67%
val set=====>loss: 0.3533   acc: 51.85%



100%|██████████| 755/755 [02:12<00:00,  5.72it/s]

Adjusting learning rate of group 0 to 2.5000e-05.


epoch 40%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.3387   acc: 53.10%
val set=====>loss: 0.3510   acc: 51.72%



# generate submission

In [ ]:
import os
import numpy as np
import math
from tqdm import tqdm

# set the data augmentation of testing data
test_trans = transforms.Compose([
            transforms.Resize((400, 400), InterpolationMode.BILINEAR),
            transforms.RandomCrop((229, 229)),

            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)
# set the model you want to use
inc = models.inception_v3(pretrained=True)
for param in inc.parameters():
    param.requires_grad = True
inc.fc = nn.Linear(2048, 8)

# generate submission files
# please change n, m to the number you wanna use 
# for ensembling the prediction
n = 3
m = 10
for i in range(n, m):
    INFERENCE = {
        'model': inc,
        'weights_path': '/content/drive/MyDrive/VRDL/final_project3/inception_v3/sgd0.0001_cos.pth',
        'test_data_dir':'/content/data/fish/test',
        'save_sub_path':f'/content/drive/MyDrive/VRDL/final_project3/inception_v3/submit{i}.csv',
        'num_augmentation':i,
        'test_trans':test_trans,
        'batch_size':32
    }

    ft.inference(**INFERENCE)


Loading model and weights from training process ...
0th augmentation for testing ...


100%|██████████| 412/412 [03:40<00:00,  1.87it/s]


1th augmentation for testing ...


100%|██████████| 412/412 [03:41<00:00,  1.86it/s]


2th augmentation for testing ...


 80%|███████▉  | 329/412 [02:57<00:44,  1.86it/s]


KeyboardInterrupt: ignored